In [144]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from datetime import datetime, timedelta

import pm4py
from pm4py.util.xes_constants import DEFAULT_TIMESTAMP_KEY 

def simplify(graph):
    to_remove = set()
    for event in graph['events']:
        has_relations = False
        for other in graph['events']:
            a = other in graph['conditionsFor'].get(event, set())
            b = event in graph['conditionsFor'].get(other, set())
            c = other in graph['responseTo'].get(event, set())
            d = event in graph['responseTo'].get(other, set())
            e = other in graph['excludesTo'].get(event, set())
            f = event in graph['excludesTo'].get(other, set())
            if a or b or c or d or e or f:
                has_relations = True
                break

        if not has_relations:
            to_remove.add(event)

    graph['events'] -= to_remove
    graph['labels'] -= to_remove
    graph['labelMapping'] = {event: event for event in graph['events']}

def metrics(graph):
    pos = pm4py.conformance_dcr(log[log['case:label'] == "Required"], graph)
    neg = pm4py.conformance_dcr(log[log['case:label'] == "Forbidden"], graph)
    TP = sum(1 for trace in pos if trace['is_fit'])
    FN = sum(1 for trace in pos if not trace['is_fit'])
    TN = sum(1 for trace in neg if not trace['is_fit'])
    FP = sum(1 for trace in neg if trace['is_fit'])
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / (TP + FP + TN + FN) if (TP + FP + TN + FN) > 0 else 0
    
    print('True Positive: ', TP)
    print('False Negative:', FN)
    print('True Negative: ', TN)
    print('False Positive:', FP)
    print('Accuracy: ', accuracy)
    print('Precision:', precision)
    print('Recall:   ', recall)
    print('F1-Score: ', f1_score)


# Other miners and pm4py.conformance_dcr typicaly require a timestamp column.
# This is a weird forced requirement, that is rearely used, but often enfored :|
log = pm4py.read_xes("./Dreyers Foundation.xes")
if True:
    if not DEFAULT_TIMESTAMP_KEY in log.columns:
        date = datetime(2024, 1, 1, 0, 0, 0)
        timestamp = pd.Series([date + timedelta(minutes=i) for i in range(len(log))])
        log[DEFAULT_TIMESTAMP_KEY] = pd.to_datetime(timestamp)

# Choose whether to run our implementation or the DisCoveR method.
if True:
    graph = pm4py.discover_dcr_rejection(log, "Required", "Forbidden", seed=42)
else:
    graph, _ = pm4py.discover_dcr(log[log['case:label'] == 'Required'])

metrics(graph)
simplify(graph)
pm4py.view_dcr(graph)

[autoreload of networkx.classes.graphviews failed: Traceback (most recent call last):
  File "/home/chris/miniconda3/envs/gds/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/chris/miniconda3/envs/gds/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/chris/miniconda3/envs/gds/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/chris/miniconda3/envs/gds/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 309, in update_function
    setattr(old, name, getattr(new, name))
ValueError: subgraph_view() requires a code object with 5 free vars, not 0
]
[autoreload of networkx.generators.expanders failed: Traceback (most recent call last):
  File "/home/chris/miniconda3/envs/gds/lib/python3.10/site-packages/IPython/extensions/au

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


parsing log, completed traces ::   0%|          | 0/700 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/chris/miniconda3/envs/gds/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_132914/3486901750.py", line 67, in <module>
    graph = pm4py.discover_dcr_rejection(log, "Required", "Forbidden", seed=42)
  File "/mnt/c/Users/svorr/Datalogi/REBS/pm4py-dcr/pm4py/discovery.py", line 977, in discover_dcr_rejection
    return dcr_alg.apply(log, dcr_rejection, parameters=properties)
  File "/mnt/c/Users/svorr/Datalogi/REBS/pm4py-dcr/pm4py/algo/discovery/dcr_rejection/algorithm.py", line 43, in apply
    return exec_utils.get_variant(variant).apply(log, parameters=parameters)
  File "/mnt/c/Users/svorr/Datalogi/REBS/pm4py-dcr/pm4py/algo/discovery/dcr_rejection/variants/dcr_rejection.py", line 44, in apply
    return disc.mine(log, parameters = parameters)
  File "/mnt/c/Users/svorr/Datalogi/REBS/pm4py-dcr/pm4py/algo/discovery/dcr_reject